In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2025-10-20 11:00 ajna         INFO     Configuração de log efetuada


2025-10-20 11:00:58,765 ajna         INFO     Configuração de log efetuada


In [3]:
%load_ext autoreload 
%autoreload 2
engine = create_engine(SQL_URI)

# Recuperar informações de REDEX e OPERADORES

In [4]:
# Constantes organizando códigos
LISTA_OPERADORES = "('8931356', '8931359', '8931404', '8931318')"  # Terminais exportadores BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO
LISTA_REDEX = "('8931309', '8933204', '8931404')" # Redex da Localfrio/Movecta, Redex da Santos Brasil Clia e Redex da DPW/EMBRAPORT
FILTRO_LIKE_REDEX = "'89327%'"  # Abrange todos os outros Redex santistas

from datetime import datetime, timedelta

agora = datetime.now()
ha_X_dias = agora - timedelta(days=7)

# Parâmetros de data
data_final = agora.strftime('%Y-%m-%d %H:%M:%S')
data_inicial = ha_X_dias.strftime('%Y-%m-%d %H:%M:%S')

In [5]:
# SQL para entrada_operadores (alias e)
sql_entrada_operadores = f"""
SELECT 
    id, placa, placaSemirreboque, numeroConteiner, codigoRecinto, dataHoraOcorrencia, operacao, direcao, 
    cnpjTransportador, cpfMotorista, nomeMotorista, listaNfe, numeroConhecimento, vazioConteiner
FROM apirecintos_acessosveiculo
WHERE 
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'E'
    AND codigoRecinto IN {LISTA_OPERADORES}
    AND numeroConteiner !='' 
    AND vazioConteiner = 0 
"""

# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'E' é porque queremos a Entrada no Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)
# Este 'C' é porque é registro de acesso (entrada) e não agendamento
# Este '0' indica que o conteiner chegou estufado (cheio) ao Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)

In [6]:
# SQL para pesagem_operador - pesagem com recintos dos operadores (terminais exportadores)
sql_pesagem_operador = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 18000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND codigoRecinto IN {LISTA_OPERADORES}
"""
# Se o caminhao com conteiner pesa 18 toneladas ou mais, então, ele está carregado (não é vazio)

In [7]:
# SQL para pesagem_redex - pesagem com recintos Redex (estufadores)
sql_pesagem_redex = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 18000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
"""
# Se o caminhao com conteiner pesa 18 toneladas ou mais, então, ele está carregado (não é vazio)

In [8]:
# SQL com JOIN para filtrar saidas do REDEX seguidas de entrada no Terminal Exportador (Operador portuário)
sql_saida_redex = f"""
SELECT DISTINCT
    c.id,
    c.placa,
    c.placaSemirreboque,
    c.numeroConteiner,
    c.vazioConteiner,
    c.codigoRecinto,
    c.cpfMotorista,
    c.nomeMotorista,
    c.dataHoraOcorrencia,
    c.direcao
FROM apirecintos_acessosveiculo AS c
JOIN apirecintos_acessosveiculo AS e
    ON e.numeroConteiner = c.numeroConteiner 
    AND e.placaSemirreboque = c.placaSemirreboque
    AND e.dataHoraOcorrencia >= c.dataHoraOcorrencia
    AND e.direcao = 'E'
    AND e.operacao = 'C'
    AND (
        (e.codigoRecinto NOT IN {LISTA_REDEX} AND e.codigoRecinto NOT LIKE {FILTRO_LIKE_REDEX})
        OR e.codigoRecinto = '8931404'
    )
    AND e.codigoRecinto != c.codigoRecinto
WHERE 
    c.dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND c.operacao = 'C'
    AND c.direcao = 'S'
    AND (c.codigoRecinto IN {LISTA_REDEX} OR c.codigoRecinto LIKE {FILTRO_LIKE_REDEX})
    AND c.numeroConteiner != ''
"""
# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'S' é porque queremos a Saida do Redex estufador
# Este 'C' é porque é registro de acesso (saída) e não agendamento
# A tabela c. é a tabela de ACESSO DE SAÍDA do REDEX no API RECINTOS
# A tabela e. é a tabela de ACESSO DE ENTRADA no TERMINAL EXPORTADOR (Operador Portuário)
# A lógica é que o caminhão com conteiner SAI DO REDEX ANTES DE ENTRAR NO TERMINAL EXPORTADOR (por isso o maior igual)
# Atenção! A DPW/EMBRAPORT pode operar como REDEX e envia carga para a BTP como pode receber carga de um REDEX e operar como Terminal exportador
# Este código contempla estes dois casos da DPW/EMBRAPORT

In [9]:
# Execução das consultas com parâmetros
df_entrada_operadores = pd.read_sql(
    text(sql_entrada_operadores), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [10]:
df_entrada_operadores

,id,placa,placaSemirreboque,numeroConteiner,codigoRecinto,dataHoraOcorrencia,operacao,direcao,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento,vazioConteiner
0,13081490,AIX0I55,AGD5B72,MSNU8547400,8931318,2025-10-13 11:12:52,C,E,32256956000145,03845418877,MARCOS ANTONIO RIBEIRO GONCALVES,,,0
1,13081496,BXB6I15,ESU6B22,MRSU4258427,8931318,2025-10-13 11:18:48,C,E,01896217000194,37848634850,LEANDRO RICARDO FERREIRA DA SILVA,,,0
2,13081505,EIX7231,CUD4906,BEAU2965397,8931318,2025-10-13 11:30:34,C,E,12014047000193,31146520425,ALCIDES FEITOZA DA SILVA FILHO,,,0
3,13081510,FRS2D82,AIK5928,MSBU5449018,8931318,2025-10-13 11:36:32,C,E,12014047000193,11945072806,MARCOS CESAR DE MENDONCA,,152505308991477,0
4,13081516,ALN2I78,FEI2183,MRSU5473609,8931318,2025-10-13 11:45:52,C,E,32256956000145,07986462841,DEVAIR PEREIRA,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24165,13223665,NVQ5J82,ECM3I47,HAMU2390392,8931404,2025-10-19 20:08:43,C,E,01896217000194,19230715808,UBIRATAN SCHEK,,,0
24166,13223824,FZT0H26,FST8945,TLLU8274977,8931404,2025-10-19 20:23:57,C,E,58180316000192,29986271886,FLAVIO EDUARDO SARAIVA PEREIRA,,,0
24167,13223694,KMC9348,CCU1J97,TRHU4927459,8931404,2025-10-19 21:09:49,C,E,21425093000176,44779261899,LEONARDO SILVA DOS ANJOS,,,0
24168,13223698,DBM7C46,ETU5039,CMAU2646606,8931404,2025-10-19 21:16:53,C,E,21425093000176,95413790449,JEFFERSON JOSE DA SILVA LIMA,,,0


In [11]:
df_saida_redex = pd.read_sql(
    text(sql_saida_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [12]:
df_saida_redex

,id,placa,placaSemirreboque,numeroConteiner,vazioConteiner,codigoRecinto,cpfMotorista,nomeMotorista,dataHoraOcorrencia,direcao
0,13080927,JBS7C52,NXW2730,MRKU8863932,0,8931309,10919065805,MARCIO LUCIANO BATISTA DA SILVA,2025-10-13 11:12:46,S
1,13081016,RJN8I32,DAO6945,ONEU9033249,0,8931309,65257022591,LUCIANO SANTANA DE OLIVEIRA,2025-10-13 13:21:48,S
2,13102873,TKH0E49,CDR1G63,LTXU2510719,0,8931309,37091400801,FERNANDO MIRANDA FREITAS,2025-10-13 23:56:57,S
3,13103338,BWI0990,BWY7129,UETU7938017,0,8931309,31130705897,WILLIAN FARIAS DOS SANTOS,2025-10-14 14:01:35,S
4,13103603,JBW3J33,NXW2770,TEMU0469293,0,8931309,25715775817,CARLOS ANDRE SANTOS DE JESUS,2025-10-14 22:45:01,S
...,...,...,...,...,...,...,...,...,...,...
7114,13234491,KHX8E39,BTR2E92,RGTU1001046,0,8933204,39260512808,JEFFERSON PIRES SANTOS,2025-10-19 13:51:01,S
7115,13234586,FEI3653,FEI4855,RGTU1003970,0,8933204,26194920840,ALUISIO PEREIRA DA SILVA,2025-10-19 16:10:23,S
7116,13234595,APB3D48,BWI1H04,HAMU1999804,0,8933204,40788157825,MATEUS CARDOSO DOS SANTOS,2025-10-19 17:54:53,S
7117,13234608,FEI3653,FEI4855,MSDU6768995,0,8933204,16234893856,MICHEL DOS SANTOS RIBEIRO,2025-10-19 19:25:20,S


In [13]:
df_saidas_entradas = pd.merge(df_entrada_operadores, df_saida_redex, how='inner', on=['placaSemirreboque', 'numeroConteiner'] , suffixes=('_E_Operador', '_S_REDEX'))
# Este 'inner' faz com que a tabela final obtida saidas_entradas só tenham dados que tem em Entrada Operador e Saída Rede
# Dados que não tenham ligação correspondente (são nulos) NÃO aparecerão
# Se você quiser que os não correspondentes também apareçam troque INNER por LEFT

In [14]:
df_saidas_entradas['transitTime'] = (df_saidas_entradas['dataHoraOcorrencia_E_Operador'] - 
                                     df_saidas_entradas['dataHoraOcorrencia_S_REDEX']).dt.total_seconds() / 3600


In [15]:
# df_saidas_entradas

In [16]:
#!pip install tqdm
from tqdm.notebook import tqdm

def buscar_id_fs_files(collection, numero_conteiner, data_ocorrencia):
    filtro = {
        'metadata.numeroinformado': numero_conteiner,
        'metadata.dataescaneamento': {
            '$gte': data_ocorrencia - timedelta(hours=3),
            '$lte': data_ocorrencia + timedelta(hours=6)
        }
    }
    resultado = collection.find_one(filtro, {'_id': 1})
    if resultado:
        return resultado['_id']
    return None

# Iterar sobre as linhas do DataFrame e aplicar busca
def mapear_ids_mongo(collection, df):
    ids_fs_files = []
    cont_sucesso = 0
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Buscando ids no MongoDB"):
        numero = row['numeroConteiner']
        data = row['dataHoraOcorrencia_E_Operador']
        id_fs = buscar_id_fs_files(collection, numero, data)
        ids_fs_files.append(id_fs)
        cont_sucesso += 1 if id_fs else 0
    df['fs_files_id'] = ids_fs_files
    print(f'Registros: {len(df)}, imagens: {cont_sucesso}')

In [17]:
mapear_ids_mongo(db['fs.files'], df_saidas_entradas)

Buscando ids no MongoDB:   0%|          | 0/7019 [00:00<?, ?it/s]

Registros: 7019, imagens: 3286


In [18]:
# df_saidas_entradas.transitTime.describe()
df_saidas_entradas.rename(columns={'placa': 'placa', 'placaSemirreboque': 'placa do semireboque', 'numeroConteiner': 'contêiner', 'vazioConteiner' : 'conteinervazio',
                         'codigoRecinto_S_REDEX': 'Redex', 'dataHoraOcorrencia_S_REDEX': 'Saída REDEX','placaSemirreboque': 'placa do semireboque',
                         'codigoRecinto_E_Operador': 'Operador', 'dataHoraOcorrencia_E_Operador': 'Entrada Operador',
                         'transitTime': 'transitTime', 'fs_files_id': '_id'}, inplace=True)
df_saidas_entradas.drop(['operacao', 'direcao_E_Operador', 'listaNfe', 'numeroConhecimento', 'direcao_S_REDEX'], axis=1)
df_saidas_entradas

,id_E_Operador,placa_E_Operador,placa do semireboque,contêiner,Operador,Entrada Operador,operacao,direcao_E_Operador,cnpjTransportador,cpfMotorista_E_Operador,...,id_S_REDEX,placa_S_REDEX,vazioConteiner_S_REDEX,Redex,cpfMotorista_S_REDEX,nomeMotorista_S_REDEX,Saída REDEX,direcao_S_REDEX,transitTime,_id
0,13081531,CUA4J91,CUA5944,GCNU1364817,8931318,2025-10-13 11:56:19,C,E,02077058000169,27769375800,...,13100518,CUA4J91,0,8932797,27769375800,MARCIO JOSE CARDOSO DOS SANTOS,2025-10-13 11:10:33,S,0.762778,68ee365ecca6eeddad8167bd
1,13081541,CUA7A41,CUA5J36,GCNU1374266,8931318,2025-10-13 12:04:23,C,E,02077058000169,19752170803,...,13100521,CUA7A41,0,8932797,19752170803,MARCELO MORGADO,2025-10-13 11:12:15,S,0.868889,68ee365ecca6eeddad8167c1
2,13081551,DVS1I68,CUA5F95,ACLU2795184,8931318,2025-10-13 12:12:57,C,E,02077058000169,37415733886,...,13100531,DVS1I68,0,8932797,37415733886,JEFFERSON DE SOUZA FELIX RODRIGUES,2025-10-13 11:27:29,S,0.757778,68ee365ecca6eeddad8167a8
3,13081561,DPC9F97,CZB8156,GCNU1308285,8931318,2025-10-13 12:21:05,C,E,02077058000169,36128313809,...,13100535,DPC9F97,0,8932797,36128313809,SHENE ANDERSON GOMES OLIVEIRA,2025-10-13 11:32:18,S,0.813056,68ee365ecca6eeddad8167b5
4,13081570,DPC9932,DPC9D37,ACLU2790109,8931318,2025-10-13 12:35:05,C,E,02077058000169,91308658400,...,13100547,DPC9932,0,8932797,91308658400,FRANCISCO RICARDO DE SOUSA ANDRADE,2025-10-13 12:04:51,S,0.503889,68ee3660cca6eeddad816969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7014,13222294,ALB3F37,EVO3H63,TCLU3011162,8931404,2025-10-18 11:11:34,C,E,06235812000100,51402254849,...,13239208,ALB3F37,0,8932792,51402254849,ANTONIO JOSE FILIPE,2025-10-18 10:00:04,S,1.191667,68f6245f0413a90d8c882838
7015,13222762,TCI6H88,UAI9F77,SUDU6211735,8931404,2025-10-18 16:55:33,C,E,20584786000149,36626391810,...,13097282,STW2G04,1,8932761,70501661620,CLAUDIO HUMBERTO ALVES FERREIRA,2025-10-13 11:05:24,S,125.835833,68f6245e0413a90d8c8827e1
7016,13222831,TCH0F51,SYN7A99,MNBU0287996,8931404,2025-10-18 18:02:34,C,E,20584786000149,00105646679,...,13097639,STP8H69,1,8932761,98454200100,JOAO BATISTA BERNARDES JUNIOR,2025-10-13 22:44:29,S,115.301389,68f624560413a90d8c88246d
7017,13223100,SEQ7G98,GAG0079,MNBU4437203,8931404,2025-10-18 22:03:12,C,E,02916265009116,93464541649,...,13097483,SEX1A95,1,8932761,07250508610,TACIANO ANDRADE VILELA,2025-10-13 17:52:14,S,124.182778,None


In [19]:
from IPython.display import display, HTML

def get_outliers(df):
    mean = df['transitTime'].mean()
    std = df['transitTime'].std()
    outliers = df[(df['transitTime'] < mean - 1 * std) | (df['transitTime'] > mean + 1 * std)]
    return outliers

def generate_html(_id):
    url = f"https://ajna1.rfoc.srf/virasana/file?_id={_id}"
    img = f"https://ajna1.rfoc.srf/virasana/imagens_cmap?_id={_id}"
    return f'<a href="{url}"><img src="{img}" width="200"></a>'
    
outliers = get_outliers(df_saidas_entradas)[['placa do semireboque', 'contêiner', 'Redex', 'Saída REDEX',
                                  'Operador', 'Entrada Operador', 'transitTime', '_id']]

outliers['html'] = outliers['_id'].apply(generate_html)
display(HTML(outliers.to_html(escape=False)))

,placa do semireboque,contêiner,Redex,Saída REDEX,Operador,Entrada Operador,transitTime,_id,html
1789,BXE9H02,HAMU2948421,8931404,2025-10-15 19:49:00,8931356,2025-10-16 10:38:50,14.830556,None,
2717,DBM6I81,HAMU3830768,8932796,2025-10-16 19:18:47,8931356,2025-10-17 15:11:46,19.883056,None,
2846,GII2I83,HAMU4542215,8932775,2025-10-17 09:26:29,8931356,2025-10-17 17:30:00,8.058611,None,
2953,FZQ2G08,MRSU7458780,8931404,2025-10-15 20:07:30,8931356,2025-10-17 22:38:43,50.520278,None,
2990,BXE6G09,FBIU5452336,8932762,2025-10-17 13:37:00,8931356,2025-10-18 00:14:19,10.621944,None,
2991,BXE6G09,FBIU5452336,8932762,2025-10-17 13:37:20,8931356,2025-10-18 00:14:19,10.616389,None,
3053,FVZ3070,UETU6235724,8931404,2025-10-17 00:32:29,8931356,2025-10-18 03:49:49,27.288889,None,
3265,SUP3A77,TCLU1702170,8931404,2025-10-17 04:48:41,8931356,2025-10-18 14:33:19,33.743889,None,
3286,GFH7D33,CAAU4136632,8932761,2025-10-15 15:11:52,8931356,2025-10-18 17:10:52,73.983333,None,
3296,SUZ2E22,BEAU5836520,8931404,2025-10-14 18:52:41,8931356,2025-10-18 19:38:29,96.763333,None,


In [20]:
# Adicionar células clicáveis que levem ao Ajna
df_saidas_entradas['contêiner'] = df_saidas_entradas['contêiner'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/bhadrasana2/consulta_container?numerolote={x}", "{x}")'
)
df_saidas_entradas['_id'] = df_saidas_entradas['_id'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/virasana/file?_id={x}", "{x}")'
)

# df_saidas_entradas

In [21]:
df_saidas_entradas.to_excel('saidas_entradas.xlsx', index=False)

In [22]:
df_pesagem_operador = pd.read_sql(
    text(sql_pesagem_operador), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

df_pesagem_redex = pd.read_sql(
    text(sql_pesagem_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)



# Seguem os merges e filtros conforme a lógica desejada especificada anteriormente
# Por exemplo, aqui unimos pesagem_redex com saida_redex e depois com entrada_operadores conforme passo a passo
df_pesagem_saida = pd.merge(df_pesagem_redex, df_saida_redex, on=['placaSemirreboque', 'codigoRecinto', 'numeroConteiner'], suffixes=('_pesagem', '_saida'))

df_pesagem_saida = df_pesagem_saida[
    (df_pesagem_saida['dataHoraOcorrencia_pesagem'] <= df_pesagem_saida['dataHoraOcorrencia_saida']) &
    ((df_pesagem_saida['dataHoraOcorrencia_saida'] - df_pesagem_saida['dataHoraOcorrencia_pesagem']).dt.days <= 7) &
    (df_pesagem_saida['direcao'] == 'S')
]

df_final = pd.merge(df_pesagem_saida, df_entrada_operadores, left_on=['placaSemirreboque', 'numeroConteiner'], right_on=['placaSemirreboque', 'numeroConteiner'], suffixes=('', '_entrada'))

df_final = df_final[
    ((df_final['dataHoraOcorrencia'] - df_final['dataHoraOcorrencia_saida']).dt.days <= 7) &
    (df_final['codigoRecinto'] != df_final['codigoRecinto_entrada']) 
]

df_final = df_final.drop_duplicates()

# Seleção de colunas para output
output_cols = [
    'dataHoraOcorrencia_pesagem', 'id_pesagem', 'placa', 'placaSemirreboque', 'pesoBrutoBalanca', 'numeroConteiner', 'codigoRecinto_pesagem',
    'dataHoraOcorrencia_saida', 'placa', 'codigoRecinto_saida', 'direcao', 'numeroConteiner',
    'dataHoraOcorrencia', 'placa', 'placaSemirreboque', 'id', 'cnpjTransportador', 'cpfMotorista', 'nomeMotorista', 'codigoRecinto', 'operacao', 'direcao', 'numeroConteiner', 'listaNfe', 'numeroConhecimento'
]

df_result = df_final[output_cols]

print(df_result.head())

KeyError: "['codigoRecinto_pesagem', 'codigoRecinto_saida'] not in index"